In [ ]:
# staging.ipynb
# ----------------------
# 1. Подключение к PostgreSQL
# ----------------------
import pandas as pd
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os

# Загружаем переменные окружения
load_dotenv()

DB_USER = os.getenv("POSTGRES_USER")
DB_PASSWORD = os.getenv("POSTGRES_PASSWORD")
DB_HOST = os.getenv("POSTGRES_HOST")
DB_PORT = os.getenv("POSTGRES_PORT")
DB_NAME = os.getenv("POSTGRES_DB")
SCHEMA = os.getenv("POSTGRES_SCHEMA")

# Создаём движок SQLAlchemy
engine = create_engine(f"postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}")

# Функция для чтения таблицы из схемы analytics
def read_table(table_name: str):
    query = f'SELECT * FROM {SCHEMA}.{table_name};'
    return pd.read_sql(query, engine)

In [20]:
# ----------------------
# 2. Читаем сырые таблицы
# ----------------------

df_users = read_table("users")
df_sessions = read_table("sessions")
df_subs = read_table("subscriptions")
df_invoices = read_table("invoices")
df_plans = read_table("plans")
df_events = read_table("events")

# Быстрый просмотр
print("Users:", df_users.shape)
print("Sessions:", df_sessions.shape)
print("Subscriptions:", df_subs.shape)
print("Invoices:", df_invoices.shape)
print("Plans:", df_plans.shape)
print("Events:", df_events.shape)

# Размер данных
print(f"Размер DataFrame: {df_users.shape}")
print(f"Количество строк: {df_users.shape[0]}")
print(f"Количество столбцов: {df_users.shape[1]}")

# Общая информация
df_events.info()

# Просмотр первых и случайных строк
print(f'Просмотр первых и случайных строк {df_users.columns}')

display(df_users.head())
display(df_users.sample(5))

Users: (2000, 3)
Sessions: (5914, 7)
Subscriptions: (658, 5)
Invoices: (2167, 9)
Plans: (3, 4)
Events: (16251, 3)
Размер DataFrame: (2000, 3)
Количество строк: 2000
Количество столбцов: 3
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16251 entries, 0 to 16250
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   user_id     16251 non-null  int64 
 1   event_date  16251 non-null  object
 2   event_name  16251 non-null  object
dtypes: int64(1), object(2)
memory usage: 381.0+ KB
Просмотр первых и случайных строк Index(['user_id', 'signup_date', 'region'], dtype='object')


,user_id,signup_date,region
0,1,2024-04-12,Казань
1,2,2024-03-15,Москва
2,3,2024-02-22,Новосибирск
3,4,2024-02-27,Казань
4,5,2024-01-15,Москва


,user_id,signup_date,region
512,513,2024-05-17,Нижний Новгород
1095,1096,2024-03-20,Москва
294,295,2024-05-17,Санкт-Петербург
833,834,2024-04-20,Казань
81,82,2024-05-20,Казань
